# Импортируем библиотеки
- `numpy`, `matplotlib` - математическая библиотека и визуализация
- `torch` (`torchvision`) - библиотека глубокого обучения

In [73]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms,datasets
from torchvision.utils import save_image
from PIL import Image

%matplotlib inline

### Создадим массив со значениями скрытого слоя

In [75]:
hidden_sizes = [10, 20, 30, 50, 70, 100]

# Машина Больцмана

### Создадим единичную машину Больцмана

Сделаем запись пары input/output для каждого значения скрытого слоя в папку **/images_rbm1**

<image src="https://images.velog.io/images/chulhongsung/post/f5ce5a1c-d7e9-41f9-a888-9b8ac700d95b/The-basic-structure-of-restricted-Boltzmann-machine-RBM.png">

In [76]:
for hidden_size in hidden_sizes:
    class RBM(nn.Module):
        def __init__(self,h1):
            super(RBM, self).__init__()
            self.rbm1 = nn.Sequential(nn.Linear(784, h1), nn.Sigmoid())
            self.rbm6 = nn.Sequential(nn.Linear(h1, 784), nn.Sigmoid())
            
        def forward(self, x):
            h1 = self.rbm1(x)
            h6 = self.rbm6(h1)
            return h6

    transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    trainset = torchvision.datasets.MNIST(
        root='./data', 
        train=True, 
        download=True,
        transform=transform
        )
    
    trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=64,
        shuffle=True
        )

    criterion = nn.MSELoss()
    rbm = RBM(hidden_size)
    optimizer = optim.Adam(rbm.parameters(), lr=0.01)

    num_epochs = 10
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for data in trainloader:
            inputs, _ = data
            inputs = inputs.view(-1, 784)
            
            optimizer.zero_grad()
            outputs = rbm(inputs)
            
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        print('Epoch [%d/%d], Loss: %.4f' % (epoch + 1, num_epochs, running_loss / len(trainloader)))

    testset = torchvision.datasets.MNIST(
        root='./data',
        train=False,
        download=True,
        transform=transform
        )
    
    testloader = torch.utils.data.DataLoader(
        testset,
        batch_size=1,
        shuffle=True
        )
    
    with torch.no_grad():
        for data in testloader:
            inputs, _ = data
            inputs = inputs.view(-1, 784)
            outputs = rbm(inputs)
            
            inputs = inputs.view(1, 28, 28)
            outputs = outputs.view(1, 28, 28)
            
            torchvision.utils.save_image(inputs, './images_rbm1/input_hidden_size_{}_.png'.format(hidden_size))
            torchvision.utils.save_image(outputs, './images_rbm1/output_hidden_size_{}_.png'.format(hidden_size))
            break

Epoch [1/10], Loss: 0.0693
Epoch [2/10], Loss: 0.0675
Epoch [3/10], Loss: 0.0675
Epoch [4/10], Loss: 0.0675
Epoch [5/10], Loss: 0.0675
Epoch [6/10], Loss: 0.0675
Epoch [7/10], Loss: 0.0675
Epoch [8/10], Loss: 0.0675
Epoch [9/10], Loss: 0.0675
Epoch [10/10], Loss: 0.0675
Epoch [1/10], Loss: 0.0401
Epoch [2/10], Loss: 0.0236
Epoch [3/10], Loss: 0.0182
Epoch [4/10], Loss: 0.0135
Epoch [5/10], Loss: 0.0109
Epoch [6/10], Loss: 0.0088
Epoch [7/10], Loss: 0.0081
Epoch [8/10], Loss: 0.0077
Epoch [9/10], Loss: 0.0075
Epoch [10/10], Loss: 0.0074
Epoch [1/10], Loss: 0.0228
Epoch [2/10], Loss: 0.0074
Epoch [3/10], Loss: 0.0050
Epoch [4/10], Loss: 0.0041
Epoch [5/10], Loss: 0.0038
Epoch [6/10], Loss: 0.0036
Epoch [7/10], Loss: 0.0035
Epoch [8/10], Loss: 0.0034
Epoch [9/10], Loss: 0.0034
Epoch [10/10], Loss: 0.0033
Epoch [1/10], Loss: 0.0135
Epoch [2/10], Loss: 0.0043
Epoch [3/10], Loss: 0.0028
Epoch [4/10], Loss: 0.0023
Epoch [5/10], Loss: 0.0021
Epoch [6/10], Loss: 0.0020
Epoch [7/10], Loss: 0.001

### Сохраним модель

In [77]:
torch.save(rbm,"./rbm1.pt")
model_rbm1 = torch.load("./rbm1.pt")

### Обученим модель из трех машин Больцмана

Сделаем запись пары input/output для каждого значения скрытого слоя в папку **/images_rbm3**

In [ ]:
for hidden_size in hidden_sizes:
    class DBN(nn.Module):
        def __init__(self,h1,h_last):
            super(DBN, self).__init__()
            
            self.rbm1 = nn.Sequential(nn.Linear(784, h1), nn.Sigmoid())
            self.rbm2 = nn.Sequential(nn.Linear(h1, h1), nn.Sigmoid())
            self.rbm3 = nn.Sequential(nn.Linear(h1, h_last), nn.Sigmoid())
            self.rbm4 = nn.Sequential(nn.Linear(h_last, h1), nn.Sigmoid())
            self.rbm5 = nn.Sequential(nn.Linear(h1, h1), nn.Sigmoid())
            self.rbm6 = nn.Sequential(nn.Linear(h1, 784), nn.Sigmoid())
            
        def forward(self, x):
            h1 = self.rbm1(x)
            h2 = self.rbm2(h1)
            h3 = self.rbm3(h2)
            h4 = self.rbm4(h3)
            h5 = self.rbm5(h4)
            h6 = self.rbm6(h5)
            
            return h6

    transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    trainset = torchvision.datasets.MNIST(
        root='./data',
        train=True,
        download=True,
        transform=transform
        )
    trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=64,
        shuffle=True
        )

    criterion = nn.MSELoss()
    dbn = DBN(hidden_size,500)
    optimizer = optim.Adam(dbn.parameters(), lr=0.001)

    num_epochs = 10
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for data in trainloader:
            inputs, _ = data
            inputs = inputs.view(-1, 784)
            optimizer.zero_grad()
            outputs = dbn(inputs)
            
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, running_loss/len(trainloader)))

    testset = torchvision.datasets.MNIST(
        root='./data',
        train=False,
        download=True,
        transform=transform
        )
    testloader = torch.utils.data.DataLoader(
        testset,
        batch_size=1,
        shuffle=True
        )
    
    with torch.no_grad():
        for data in testloader:
            inputs, _ = data
            
            inputs = inputs.view(-1, 784)
            outputs = dbn(inputs)
            
            inputs = inputs.view(1, 28, 28)
            outputs = outputs.view(1, 28, 28)
            
            torchvision.utils.save_image(inputs, './images_rbm3/input_hidden_size_{}_.png'.format(hidden_size))
            torchvision.utils.save_image(outputs, './images_rbm3/output_hidden_size_{}_.png'.format(hidden_size))
            
            break

### Сохраним модель

In [ ]:
torch.save(dbn,"./rbm3.pt")
model_rbm3 = torch.load("./rbm3.pt")

### Протестируем **первую** модель на 10 изображениях из датасета `FashionMNIST`
Запишем результаты input/output в папку **/fashion_mnist_test_rbm1**

In [ ]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
testset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
    )
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=1,
    shuffle=True
    )

with torch.no_grad():
    for i, (inputs, _) in enumerate(testloader):
        if i == 10:
            break
        inputs = inputs.view(-1, 784)
        outputs = model_rbm1(inputs)
        
        inputs = inputs.view(1, 28, 28)
        outputs = outputs.view(1, 28, 28)
        
        torchvision.utils.save_image(inputs, f'./fashion_mnist_test_rbm1/input_{i}.png')
        torchvision.utils.save_image(outputs, f'./fashion_mnist_test_rbm1/output_{i}.png')

### Протестируем **вторую** модель на 10 изображениях из датасета `FashionMNIST`
Запишем результаты input/output в папку **/fashion_mnist_test_rbm3**

In [84]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
testset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
    )
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=1,
    shuffle=True
    )

with torch.no_grad():
    for i, (inputs, _) in enumerate(testloader):
        if i == 10:
            break
        
        inputs = inputs.view(-1, 784)
        outputs = model_rbm3(inputs)
        
        inputs = inputs.view(1, 28, 28)
        outputs = outputs.view(1, 28, 28)
        
        torchvision.utils.save_image(inputs, f'./fashion_mnist_test_rbm3/input_{i}.png')
        torchvision.utils.save_image(outputs, f'./fashion_mnist_test_rbm3/output_{i}.png')

# Результат

- По результатам исследования минимальный размер скрытого слоя для удовлетворительного представления изображений `28x28` пикселей составил `500` скрытых нейронов
- При таком значении на обучающем датасете и на тестировании `FashionMNIST` обе модели справляются с удовлетворительным восстановлением изображений `28x28` пикселей